In [ ]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
def ScheduleCrawling():

    driver = webdriver.Chrome('/Users/kanovatio/Downloads/chromedriver 3')
    driver.implicitly_wait(2)

    driver.get("https://www.airport.co.kr/gimpo/cms/frCon/index.do?MENU_ID=1060")
    time.sleep(1)


    final_code_list = []
    final_code_list2 = []

    input_year = input('몇 년도의 운항 스케줄을 뽑으시겠습니까? ')

    if input_year == '2018':
        year_len = 4
    elif input_year == '2019':
        year_len = 3
    elif input_year == '2020':
        year_len = 2
    elif input_year == '2021':
        year_len = 1
    elif input_year == '2022':
        year_len = 0
    else:
        print('연도를 잘못 입력했습니다. ')
        return False

    if year_len == 0:
        month_len = 6
    else:
        month_len = 12
    driver.find_element(By.CSS_SELECTOR, '#sendForm > fieldset > div:nth-child(1) > span.inpGrp > span > label').click()

    # 제주 선택
    driver.find_element(By.CSS_SELECTOR, '#sendForm > fieldset > div:nth-child(3) > select > option:nth-child(7)').click()

    time.sleep(2.0)
    driver.find_element(By.CSS_SELECTOR, '#fd-but-datepicker1').click()
    time.sleep(0.5)



    for j in range(year_len):
        driver.find_element(By.CSS_SELECTOR, '#datepicker1-prev-year-but').click()

    for page in range(10):
        driver.find_element(By.CSS_SELECTOR, '#datepicker1-prev-month-but').click()

    print('################ 항공 스케줄 크롤링을 시작합니다. ##########################')
    time.sleep(1.0)
    # 12월까지
    for num in range(month_len):
        print(f'{input_year}년 {num+1}월의 항공 스케줄 데이터를 추출하고 있습니다.')
        time.sleep(0.5)
        # 2, 3, 4 주의 첫날만 클릭하자
        for i in range(2, 5):
            driver.find_element(By.CSS_SELECTOR, '#fd-datepicker1 > div > table > tbody > tr:nth-child({}) > td:nth-child(1)'.format(i)).click()
            time.sleep(1.0)

            driver.find_element(By.CSS_SELECTOR, '#sendForm > fieldset > button').click()
            time.sleep(0.5)



            # 출발기준
            time.sleep(2)
            code_list = driver.find_elements(By.CSS_SELECTOR, '#outTbody > tr')

            print('#######{}년 {}월 {}주차 데이터셋 추출 시작(출발기준) #######'.format(input_year, num+1, i))
            for code in code_list:
                # 항공편
                air_num = code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.name > a').text.split('(')[1].split(')')[0]


                airline_name = code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.name > a').text.split('(')[0]

                start_loc =  code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.airroute').text.split('에서')[0]

                end_loc =  code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.airroute').text.split('에서')[1]



                start_time =  code.find_element(By.CSS_SELECTOR, 'td.start > p').text

                end_time = code.find_element(By.CSS_SELECTOR, 'td:nth-child(3) > p').text

                # 운항 기간
                run_time = code.find_element(By.CSS_SELECTOR, 'td:nth-child(11) > p').text.split('~')[0].rstrip()


                print([air_num, airline_name,start_loc, end_loc, start_time, end_time, run_time])
                print()
                final_code_list.append([air_num, airline_name ,start_loc, end_loc, start_time, end_time, run_time])



            # 도착 기준
            code_list2 = driver.find_elements(By.CSS_SELECTOR, '#inTbody > tr')
            print('#######{}년 {}월 {}주차 데이터셋 추출 시작(도착 기준) #######'.format(input_year, num+1, i))

            for code in code_list2:
                # 항공편
                air_num = code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.name > a').text.split('(')[1].split(')')[0]


                airline_name = code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.name > a').text.split('(')[0]

                start_loc =  code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.airroute').text.split('에서')[0]

                end_loc =  code.find_element(By.CSS_SELECTOR, 'td:nth-child(1) > p > span.airroute').text.split('에서')[1]


                start_time =  code.find_element(By.CSS_SELECTOR, 'td.start > p').text

                end_time = code.find_element(By.CSS_SELECTOR, 'td:nth-child(3) > p').text

                # 운항 기간
                run_time = code.find_element(By.CSS_SELECTOR, 'td:nth-child(11) > p').text.split('~')[0].rstrip()


                print([air_num, airline_name, start_loc, end_loc, start_time, end_time, run_time])
                print()
                final_code_list2.append([air_num, airline_name, start_loc, end_loc, start_time, end_time, run_time])

            time.sleep(2.0)
            driver.find_element(By.CSS_SELECTOR, '#fd-but-datepicker1').click()
            print('-------------------------------------------')
            time.sleep(0.5)

        driver.find_element(By.CSS_SELECTOR, '#datepicker1-next-month-but').click()

        #fd-datepicker1 > div > table > tbody > tr:nth-child(2)
    print('############## {}년 {}월 까지의 운항 스케줄 크롤링을 종료합니다. #####################'.format(input_year, num+1))
    driver.quit()
    # 출발 기준
    final_code_df = pd.DataFrame(final_code_list, columns=['운항편명', '항공사명', '출발공항', '도착공항', '출발예정시간', '도착예정시간', '운항일자'])

    # 도착기준
    final_code_df2 = pd.DataFrame(final_code_list2, columns=['운항편명', '항공사명', '출발공항', '도착공항', '출발예정시간', '도착예정시간', '운항일자'])

    # 중복 제거
    final_code_df.drop_duplicates(keep='first', ignore_index=True, inplace=True)

    # 운항일자순으로 정렬
    final_code_df.sort_values(by='운항일자', ascending=False, inplace=True)

    # 중복 제거
    final_code_df2.drop_duplicates(keep='first', ignore_index=True, inplace=True)

    # 운항일자순으로 정렬
    final_code_df2.sort_values(by='운항일자', ascending=False, inplace=True)




    return final_code_df, final_code_df2, input_year




In [ ]:
# 위에 함수 안에 크롬 드라이버 경로 자기 경로로 바꾸주자
final_code_df, final_code_df2, input_year = ScheduleCrawling()

/var/folders/9t/s4kfwbj54wj1sw01v7vtb68h0000gn/T/ipykernel_14247/2589896660.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/Users/kanovatio/Downloads/chromedriver 3')


몇 년도의 운항 스케줄을 뽑으시겠습니까? 2022
################ 항공 스케줄 크롤링을 시작합니다. ##########################
2022년 1월의 항공 스케줄 데이터를 추출하고 있습니다.
#######2022년 1월 2주차 데이터셋 추출 시작(출발기준) #######
['7C163', '제주항공', '서울/김포', '제주', '06:00', '07:05', '2022.01.01']

['OZ8901', '아시아나항공', '서울/김포', '제주', '06:05', '07:10', '2021.10.31']

['OZ8903', '아시아나항공', '서울/김포', '제주', '06:10', '07:20', '2021.10.31']

['RS931', '에어서울', '서울/김포', '제주', '06:10', '07:10', '2022.01.01']

['OZ8905', '아시아나항공', '서울/김포', '제주', '06:20', '07:30', '2021.10.31']

['7C101', '제주항공', '서울/김포', '제주', '06:30', '07:40', '2021.12.30']

['LJ303', '진에어', '서울/김포', '제주', '06:30', '07:45', '2021.10.31']

['LJ401', '진에어', '서울/김포', '제주', '06:35', '07:45', '2022.01.01']

['TW701', '티웨이항공', '서울/김포', '제주', '06:40', '07:50', '2021.10.31']

['7C151', '제주항공', '서울/김포', '제주', '06:45', '07:55', '2021.10.31']

['RS901', '에어서울', '서울/김포', '제주', '06:45', '07:45', '2021.11.02']

['RS901', '에어서울', '서울/김포', '제주', '06:45', '07:55', '2021.11.07']

['OZ8907', '아시아나항공', '서울/김포', '

['7C145', '제주항공', '서울/김포', '제주', '16:45', '17:55', '2021.10.31']

['7C145', '제주항공', '서울/김포', '제주', '16:55', '18:05', '2021.10.31']

['KE1257', '대한항공', '서울/김포', '제주', '17:05', '18:15', '2021.12.06']

['7C165', '제주항공', '서울/김포', '제주', '17:15', '18:25', '2022.01.01']

['KE1259', '대한항공', '서울/김포', '제주', '17:15', '18:25', '2021.10.31']

['BX8095', '에어부산', '서울/김포', '제주', '17:20', '18:30', '2022.01.01']

['OZ8981', '아시아나항공', '서울/김포', '제주', '17:25', '18:35', '2022.01.01']

['TW725', '티웨이항공', '서울/김포', '제주', '17:25', '18:35', '2021.10.31']

['BX8019', '에어부산', '서울/김포', '제주', '17:30', '18:40', '2022.01.01']

['KE1261', '대한항공', '서울/김포', '제주', '17:35', '18:45', '2021.10.31']

['LJ323', '진에어', '서울/김포', '제주', '17:45', '19:00', '2021.10.31']

['TW9701', '티웨이항공', '서울/김포', '제주', '17:45', '18:55', '2022.01.01']

['RS907', '에어서울', '서울/김포', '제주', '17:50', '19:05', '2021.10.31']

['RS907', '에어서울', '서울/김포', '제주', '17:50', '19:10', '2021.10.31']

['BX8095', '에어부산', '서울/김포', '제주', '17:55', '19:05', '2022.01.01']


['OZ8950', '아시아나항공', '제주', '서울/김포', '14:00', '15:10', '2021.10.31']

['7C118', '제주항공', '제주', '서울/김포', '14:10', '15:20', '2021.10.31']

['LJ318', '진에어', '제주', '서울/김포', '14:15', '15:30', '2021.10.31']

['KE1240', '대한항공', '제주', '서울/김포', '14:55', '16:05', '2021.10.31']

['BX8042', '에어부산', '제주', '서울/김포', '15:00', '16:10', '2022.01.01']

['TW736', '티웨이항공', '제주', '서울/김포', '15:00', '16:10', '2021.10.31']

['7C144', '제주항공', '제주', '서울/김포', '15:05', '16:15', '2021.10.31']

['TW736', '티웨이항공', '제주', '서울/김포', '15:05', '16:15', '2021.10.31']

['KE1242', '대한항공', '제주', '서울/김포', '15:10', '16:20', '2021.10.31']

['KE1244', '대한항공', '제주', '서울/김포', '15:20', '16:30', '2021.12.06']

['KE1246', '대한항공', '제주', '서울/김포', '15:25', '16:35', '2021.12.02']

['OZ8956', '아시아나항공', '제주', '서울/김포', '15:30', '16:40', '2021.10.31']

['4H1608', '하이에어', '제주', '서울/김포', '15:40', '17:10', '2022.01.01']

['OZ8958', '아시아나항공', '제주', '서울/김포', '15:40', '16:50', '2021.10.31']

['TW9736', '티웨이항공', '제주', '서울/김포', '15:40', '16:50', '2022.0

['7C107', '제주항공', '서울/김포', '제주', '09:00', '10:10', '2021.10.31']

['KE1215', '대한항공', '서울/김포', '제주', '09:10', '10:20', '2022.01.03']

['OZ8781', '아시아나항공', '서울/김포', '제주', '09:20', '10:30', '2021.10.31']

['7C141', '제주항공', '서울/김포', '제주', '09:30', '10:40', '2021.10.31']

['TW707', '티웨이항공', '서울/김포', '제주', '09:35', '10:45', '2021.10.31']

['LJ307', '진에어', '서울/김포', '제주', '09:40', '10:50', '2022.01.01']

['OZ8751', '아시아나항공', '서울/김포', '제주', '09:40', '10:50', '2022.01.08']

['OZ8923', '아시아나항공', '서울/김포', '제주', '09:50', '10:55', '2021.10.31']

['KE1219', '대한항공', '서울/김포', '제주', '10:10', '11:20', '2021.10.31']

['RS903', '에어서울', '서울/김포', '제주', '10:25', '11:35', '2021.10.31']

['RS903', '에어서울', '서울/김포', '제주', '10:25', '11:30', '2021.10.31']

['7C135', '제주항공', '서울/김포', '제주', '10:30', '11:35', '2021.10.31']

['KE1221', '대한항공', '서울/김포', '제주', '10:30', '11:40', '2021.10.31']

['TW709', '티웨이항공', '서울/김포', '제주', '10:30', '11:40', '2021.10.31']

['TW711', '티웨이항공', '서울/김포', '제주', '10:45', '11:55', '2021.10.31

...

['7C106', '제주항공', '제주', '서울/김포', '10:10', '11:20', '2022.03.27']

['BX8090', '에어부산', '제주', '서울/김포', '10:20', '11:30', '2022.06.01']

['KE1280', '대한항공', '제주', '서울/김포', '10:20', '11:30', '2022.06.10']

['OZ8920', '아시아나항공', '제주', '서울/김포', '10:20', '11:30', '2022.03.27']

['KE1282', '대한항공', '제주', '서울/김포', '10:25', '11:35', '2022.06.10']

['KE1216', '대한항공', '제주', '서울/김포', '10:30', '11:40', '2022.06.09']

['RS922', '에어서울', '제주', '서울/김포', '10:35', '11:45', '2022.06.04']

['KE1284', '대한항공', '제주', '서울/김포', '10:40', '11:50', '2022.06.12']

['7C108', '제주항공', '제주', '서울/김포', '10:50', '12:00', '2022.03.27']

['KE1220', '대한항공', '제주', '서울/김포', '10:55', '12:05', '2022.03.27']

['KE1224', '대한항공', '제주', '서울/김포', '11:00', '12:10', '2022.06.08']

['RS904', '에어서울', '제주', '서울/김포', '11:00', '12:10', '2022.03.28']

['KE1224', '대한항공', '제주', '서울/김포', '11:05', '12:15', '2022.03.27']

['TW710', '티웨이항공', '제주', '서울/김포', '11:05', '12:15', '2022.03.27']

['OZ8918', '아시아나항공', '제주', '서울/김포', '11:10', '12:15', '2022.03.2

['RS908', '에어서울', '제주', '서울/김포', '19:45', '20:55', '2022.03.28']

['KE1272', '대한항공', '제주', '서울/김포', '19:50', '21:00', '2022.03.27']

['OZ8986', '아시아나항공', '제주', '서울/김포', '19:50', '21:00', '2022.03.27']

['7C126', '제주항공', '제주', '서울/김포', '20:00', '21:10', '2022.03.27']

['BX8024', '에어부산', '제주', '서울/김포', '20:00', '21:20', '2022.03.27']

['TW732', '티웨이항공', '제주', '서울/김포', '20:00', '21:10', '2022.06.01']

['RS908', '에어서울', '제주', '서울/김포', '20:05', '21:15', '2022.03.29']

['4H1610', '하이에어', '제주', '서울/김포', '20:10', '21:40', '2022.06.01']

['LJ320', '진에어', '제주', '서울/김포', '20:10', '21:25', '2022.03.27']

['KE1274', '대한항공', '제주', '서울/김포', '20:20', '21:30', '2022.06.09']

['OZ8992', '아시아나항공', '제주', '서울/김포', '20:20', '21:30', '2022.03.27']

['KE1270', '대한항공', '제주', '서울/김포', '20:30', '21:40', '2022.06.08']

['BX8022', '에어부산', '제주', '서울/김포', '20:40', '21:50', '2022.03.27']

['KE1270', '대한항공', '제주', '서울/김포', '20:40', '21:50', '2022.06.08']

['KE1296', '대한항공', '제주', '서울/김포', '20:40', '21:50', '2022.03.27

['KE1235', '대한항공', '서울/김포', '제주', '13:00', '14:10', '2022.03.27']

['TW717', '티웨이항공', '서울/김포', '제주', '13:05', '14:15', '2022.03.27']

['7C143', '제주항공', '서울/김포', '제주', '13:15', '14:25', '2022.03.27']

['BX8093', '에어부산', '서울/김포', '제주', '13:20', '14:30', '2022.06.01']

['KE1237', '대한항공', '서울/김포', '제주', '13:20', '14:30', '2022.03.27']

['RS905', '에어서울', '서울/김포', '제주', '13:20', '14:30', '2022.04.02']

['4H1605', '하이에어', '서울/김포', '제주', '13:25', '14:55', '2022.06.01']

['LJ311', '진에어', '서울/김포', '제주', '13:25', '14:40', '2022.06.01']

['OZ8939', '아시아나항공', '서울/김포', '제주', '13:25', '14:30', '2022.03.27']

['TW719', '티웨이항공', '서울/김포', '제주', '13:25', '14:35', '2022.03.27']

['7C119', '제주항공', '서울/김포', '제주', '13:30', '14:40', '2022.03.27']

['4H1605', '하이에어', '서울/김포', '제주', '13:35', '15:05', '2022.06.01']

['KE1239', '대한항공', '서울/김포', '제주', '13:35', '14:45', '2022.03.27']

['OZ8951', '아시아나항공', '서울/김포', '제주', '13:55', '15:05', '2022.03.27']

['TW743', '티웨이항공', '서울/김포', '제주', '13:55', '15:05', '2022.03.27

['KE1280', '대한항공', '제주', '서울/김포', '10:20', '11:30', '2022.06.10']

['OZ8920', '아시아나항공', '제주', '서울/김포', '10:20', '11:30', '2022.03.27']

['KE1282', '대한항공', '제주', '서울/김포', '10:25', '11:35', '2022.06.10']

['KE1216', '대한항공', '제주', '서울/김포', '10:30', '11:40', '2022.06.09']

['RS922', '에어서울', '제주', '서울/김포', '10:35', '11:45', '2022.06.04']

['KE1284', '대한항공', '제주', '서울/김포', '10:40', '11:50', '2022.06.12']

['7C108', '제주항공', '제주', '서울/김포', '10:50', '12:00', '2022.03.27']

['KE1220', '대한항공', '제주', '서울/김포', '10:55', '12:05', '2022.03.27']

['KE1224', '대한항공', '제주', '서울/김포', '11:00', '12:10', '2022.06.08']

['RS904', '에어서울', '제주', '서울/김포', '11:00', '12:10', '2022.03.28']

['KE1224', '대한항공', '제주', '서울/김포', '11:05', '12:15', '2022.03.27']

['TW710', '티웨이항공', '제주', '서울/김포', '11:05', '12:15', '2022.03.27']

['OZ8918', '아시아나항공', '제주', '서울/김포', '11:10', '12:15', '2022.03.27']

['7C142', '제주항공', '제주', '서울/김포', '11:15', '12:25', '2022.03.27']

['KE1226', '대한항공', '제주', '서울/김포', '11:15', '12:25', '2022.03.2

['7C126', '제주항공', '제주', '서울/김포', '20:00', '21:10', '2022.03.27']

['BX8024', '에어부산', '제주', '서울/김포', '20:00', '21:20', '2022.03.27']

['TW732', '티웨이항공', '제주', '서울/김포', '20:00', '21:10', '2022.06.01']

['RS908', '에어서울', '제주', '서울/김포', '20:05', '21:15', '2022.03.29']

['4H1610', '하이에어', '제주', '서울/김포', '20:10', '21:40', '2022.06.01']

['LJ320', '진에어', '제주', '서울/김포', '20:10', '21:25', '2022.03.27']

['KE1274', '대한항공', '제주', '서울/김포', '20:20', '21:30', '2022.06.09']

['OZ8992', '아시아나항공', '제주', '서울/김포', '20:20', '21:30', '2022.06.19']

['KE1270', '대한항공', '제주', '서울/김포', '20:30', '21:40', '2022.06.08']

['BX8022', '에어부산', '제주', '서울/김포', '20:40', '21:50', '2022.03.27']

['KE1270', '대한항공', '제주', '서울/김포', '20:40', '21:50', '2022.06.08']

['KE1296', '대한항공', '제주', '서울/김포', '20:40', '21:50', '2022.06.15']

['OZ8938', '아시아나항공', '제주', '서울/김포', '20:45', '21:55', '2022.03.27']

['TW734', '티웨이항공', '제주', '서울/김포', '20:45', '21:55', '2022.03.27']

['7C128', '제주항공', '제주', '서울/김포', '20:50', '22:00', '2022.03.27

In [ ]:
final_code_df.head()

,운항편명,항공사명,출발공항,도착공항,출발예정시간,도착예정시간,운항일자
650,OZ8981,아시아나항공,서울/김포,제주,18:25,19:35,2022.06.19
648,OZ8957,아시아나항공,서울/김포,제주,14:25,15:30,2022.06.19
649,KE1297,대한항공,서울/김포,제주,16:50,18:00,2022.06.15
647,OZ8935,아시아나항공,서울/김포,제주,10:40,11:50,2022.06.13
646,OZ8911,아시아나항공,서울/김포,제주,06:40,07:50,2022.06.13


In [ ]:
final_code_df2

,운항편명,항공사명,출발공항,도착공항,출발예정시간,도착예정시간,운항일자
660,OZ8992,아시아나항공,제주,서울/김포,20:20,21:30,2022.06.19
659,OZ8962,아시아나항공,제주,서울/김포,16:25,17:35,2022.06.19
661,KE1296,대한항공,제주,서울/김포,20:40,21:50,2022.06.15
656,KE1284,대한항공,제주,서울/김포,12:00,13:10,2022.06.15
658,KE1290,대한항공,제주,서울/김포,16:00,17:10,2022.06.13
...,...,...,...,...,...,...,...
90,OZ8958,아시아나항공,제주,서울/김포,15:40,16:50,2021.10.31
93,TW722,티웨이항공,제주,서울/김포,15:50,17:00,2021.10.31
94,LJ320,진에어,제주,서울/김포,16:00,17:10,2021.10.31
95,OZ8962,아시아나항공,제주,서울/김포,16:05,17:15,2021.10.31


In [ ]:
# 2018년일 경우 2017년 말 운항일짜가 나올 수도 있음(다른 연도도 해당) 나중에 엑셀에서 전처리 필요
# -> 만약 2019년 데이터에 운행일자가 2018년도가 나오면 해당 데이터를 2018년 데이터로 이동시키자


#  엑셀 저장 코드
final_code_df.to_excel('2022년 항공 스케줄 데이터(출발).xlsx'.format(input_year), index=False)
final_code_df2.to_excel('2022년 항공 스케줄 데이터(도착).xlsx'.format(input_year), index=False)